In [1]:
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker, Session
from sqlalchemy import create_engine, inspect, Column, Integer, String

engine = create_engine("sqlite:///db/doggo-db.sqlite")

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)



['babies', 'doggos']

In [15]:
babies = Base.classes.babies
# doggos = Base.classes.doggos

# Create our session (link) from Python to the DB
session = Session(engine)

results = session.query(babies.brth_yr, babies.gndr, babies.ethcty, babies.nm, babies.cnt, babies.rnk).all()

def FunBabies (BirthYr):
    df=pd.DataFrame(results)

    #Replace incomplete name with complete name
    df.replace('ASIAN AND PACI','ASIAN AND PACIFIC ISLANDER', inplace=True)
    df.replace('BLACK NON HISP','BLACK NON HISPANIC', inplace=True)
    df.replace('WHITE NON HISP','WHITE NON HISPANIC', inplace=True)

    #Rename FEMALE==F and MALE==M
    df.replace('FEMALE','F', inplace=True)
    df.replace('MALE','M', inplace=True)

    #Drop Duplicate rows
    dfd=df.drop_duplicates()

    #Sort by column, alphabetize names and combine "cnt" count for each name (dictionary)
    dfn=dfd[dfd['brth_yr']== BirthYr]

    count={}

    for index, row in dfn.iterrows():
        name=row["nm"]
        if name in count:
            count[name]+=row["cnt"]
        else: 
            count[name]=row["cnt"]

    #Create DF with new data (dictionary to list to DF)
    dfn_df=pd.DataFrame(list(count.items()), columns=["Name",f"Total {BirthYr} Count"])

    dfn_df_s=dfn_df.sort_values(f"Total {BirthYr} Count", ascending=False)
    return dfn_df_s

Yr2016=FunBabies(2016)
Yr2016.head()

,Name,Total 2016 Count
670,Liam,699
690,Jacob,689
662,Ethan,683
681,Noah,621
0,Olivia,559


In [16]:
Yr2015=FunBabies(2015)
Yr2015.head()

,Name,Total 2015 Count
667,Ethan,753
674,Liam,691
682,Noah,653
692,Jacob,642
0,Olivia,589


In [17]:
Yr2014=FunBabies(2014)
Yr2014.head()

,Name,Total 2014 Count
672,Ethan,728
697,Jacob,669
689,Liam,625
671,Jayden,609
708,Noah,589


In [18]:
Yr2013=FunBabies(2013)
Yr2013.head()

,Name,Total 2013 Count
645,Jayden,742
646,Ethan,688
658,Jacob,654
651,Daniel,627
0,Sophia,626


In [ ]:
for year in [2016, 2015]